In [1]:
# Linear algebra
import scipy.stats as ss
import scipy.special
from scipy import optimize
from mpmath import gamma
import numpy as np
import scipy
from scipy.stats import norm
from numpy import linalg as la
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd
from scipy.stats import multivariate_normal
#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style("whitegrid")
### plotting
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from mpl_toolkits.mplot3d import Axes3D
import cvxpy as cp
import time
### Other
import itertools
from timeit import default_timer as timer
from itertools import product
from ipywidgets import interact, widgets
import warnings
warnings.filterwarnings("ignore")

In [13]:
def LagQUAD_fourier_GBM_call_on_min_pricer(S0, K, r, T, sigma, SIGMA, N, R):
  """Compute rainbow options under the GBM model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.laguerre.laggauss(N)  # Laguerre quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = list(product(*u_1d))  # Full Tensor Isotropic Grid of abcissas
  w_ft = list(product(*w_1d))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(len(u_ft)):  # For all grid points
    u_list = laguerre_combinations(u_ft[i])  # Get combinations of Laguerre terms
    y_list = u_list + 1j * R
    phi_list = np.zeros(len(u_list), dtype=complex)  # List of characteristic function evaluations
    p_list = np.zeros(len(u_list), dtype=complex)  # List of payoff function evaluations
    g_list = np.zeros(len(u_list), dtype=complex)  # List of integrands resulting from change of variables
    for k in range(len(u_list)):  # For all possible combinations within 1 grid point
      phi_list[k] = GBM_characteristic_function(y_list[k], sigma, SIGMA, T, r)
      p_list[k] = fourier_payoff_call_on_min(y_list[k])
      g_list[k] = np.exp(np.multiply(1j, np.dot(u_list[k], X0))) * phi_list[k] * p_list[k]
    g = np.sum(g_list)
    V = V + discount * np.exp(np.sum(u_ft[i])) * np.prod(w_ft[i]) * g
  return np.real(V)  # Real Part of the integral

def LagQUAD_fourier_GBM_basket_put_pricer(S0, K, r, T, sigma, SIGMA, N, R):
  """Compute baskeyt put option under the GBM model using Gauss-Laguerre quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Laguerre quadrature.
  - R (array): Array of damping parameters.
  Returns:
  - V (float): Option price estimate
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.laguerre.laggauss(N)  # Laguerre quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = list(product(*u_1d))  # Full Tensor Isotropic Grid of abcissas
  w_ft = list(product(*w_1d))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, d*K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-np.dot(R, X0))  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(len(u_ft)):  # For all grid points
    u_list = laguerre_combinations(u_ft[i])  # Get combinations of Laguerre terms
    y_list = u_list + 1j * R
    phi_list = np.zeros(len(u_list), dtype=complex)  # array of characteristic function evaluations
    p_list = np.zeros(len(u_list), dtype=complex)  # array of payoff function evaluations
    g_list = np.zeros(len(u_list), dtype=complex)  # array of integrands resulting from the change of variables
    for k in range(len(u_list)):  # For all possible combinations within 1 grid point
      phi_list[k] = GBM_characteristic_function(y_list[k], sigma, SIGMA, T, r)
      p_list[k] = fourier_payoff_basket_put(y_list[k])
      g_list[k] = np.exp(np.multiply(1j, np.dot(u_list[k], X0))) * phi_list[k] * p_list[k]
    g = np.sum(g_list)
    V = V + discount * np.exp(np.sum(u_ft[i])) * np.prod(w_ft[i]) * g
  return np.real(V)  # Real Part of the integral

def HermQUAD_fourier_GBM_call_on_min_pricer(S0, K, r, T, sigma, SIGMA, N, R):
  """Compute call on min options under the GBM model using Gauss-Hermite quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Hermite quadrature.
  - R (array): Array of damping parameters.

  Returns:
  - V (float): Real part of the integral.
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.hermite.hermgauss(N)  # Hermite quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = np.asarray(list(product(*u_1d)))  # Full Tensor Isotropic Grid of abcissas
  w_ft = np.asarray(list(product(*w_1d)))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount_factor = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0)  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(u_ft.shape[0]):  # For all grid points
    y = u_ft[i] + 1j * R
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)
    payoff = fourier_payoff_call_on_min(y)
    V += np.exp(np.sum(u_ft[i]**2)) * np.prod(w_ft[i]) * np.exp(1j * u_ft[i] @ X0) * phi * payoff
  return np.real(discount_factor * V)  # Real Part of the integral

def HermQUAD_fourier_GBM_basket_put_pricer(S0, K, r, T, sigma, SIGMA, N, R):
  """Compute call on min options under the GBM model using Gauss-Hermite quadrature
  Args:
  - S0 (array): Initial stock prices.
  - K (float): Strike price.
  - r (float): Short rate.
  - T (float): Terminal time.
  - sigma (array): Array of standard deviations of each stock.
  - SIGMA (array): Covariance matrix.
  - N (int): Number of points per dimension for Hermite quadrature.
  - R (array): Array of damping parameters.

  Returns:
  - V (float): Real part of the integral.
  """
  d = len(S0)  # Number of stocks
  [u, w] = np.polynomial.hermite.hermgauss(N)  # Hermite quadrature nodes and weights
  u_1d = [u for i in range(d)]  # Array of 1-D abcissas for all directions
  w_1d = [w for i in range(d)]  # Array of 1-D weights for all directions
  u_ft = np.asarray(list(product(*u_1d)))  # Full Tensor Isotropic Grid of abcissas
  w_ft = np.asarray(list(product(*w_1d)))  # Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0, d*K))  # Logarithm of the ratio of initial stock prices to strike prices
  discount_factor = K * ((2*np.pi)**(-d)) * np.exp(-r*T) * np.exp(-R @ X0)  # Modified discount factor
  V = complex(0)  # Initialization of option price contract value
  for i in range(u_ft.shape[0]):  # For all grid points
    y = u_ft[i] + 1j * R
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)
    payoff = fourier_payoff_basket_put(y)
    V += np.exp(np.sum(u_ft[i]**2)) * np.prod(w_ft[i]) * np.exp(1j * u_ft[i] @ X0) * phi * payoff
  return np.real(discount_factor * V)  # Real Part of the integral

def GBM_characteristic_function(u, sigma, SIGMA, T, r):
  """Calculate the extended characteristic function of Multivariate GBM.
  Args:
  - u (array): Array of Fourier frequencies.
  - sigma (array): Array of volatilities of each stock.
  - SIGMA (array): Covariance matrix.
  - T (float): Time to maturity.
  - r (float): Risk-free interest rate.
  Returns:
  - phi (complex): Extended characteristic function value.
  """
  d = len(sigma)  # number of stocks
  phi = np.exp(np.dot(np.multiply(1j * T, u), r * np.ones(d) - 0.5 * np.diag(SIGMA)) - 0.5 * T * np.dot(u, np.dot(SIGMA, u)))
  return phi

def covariance_matrix(sigma, rho):
  """Compute the covariance matrix.
  Args:
  - sigma (array): Array of volatilities of each stock.
  - rho (array): Correlation matrix.
  Returns:
  - SIGMA (array): Covariance matrix.
  """
  SIGMA = np.dot(np.diag(sigma), np.dot(rho, np.diag(sigma)))
  return SIGMA

def fourier_payoff_call_on_min(u):
  """Compute the Fourier of the payoff of scaled (K = 1) call on min option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  denominator = (np.multiply(1j, np.sum(u)) - 1) * np.prod(np.multiply(1j, u))
  return 1 / denominator

def fourier_payoff_basket_put(u):
  """Compute the Fourier of the payoff of scaled (K = 1) basket put option.
  Args:
  - u (array): Array of Fourier frequencies.
  Returns:
  - payoff (float): Call on min option payoff Fourier transofrm value.
  """
  numerator = np.prod(scipy.special.gamma(np.multiply(-1j,u)))
  denominator = scipy.special.gamma(-1j*(np.sum(u))+2)
  return (numerator/denominator)

def laguerre_combinations(u):
  """Generate combinations of Laguerre terms due to change of variable from (-infinity,+infinity) to (0,+infinity)

  Args:
  - u (array): Laguerre terms.

  Returns:
  - Z (array): Combinations of Laguerre terms.
  """
  d = len(u)  # Dimensionality
  aux = np.array(u) * (-1)  # Negate each element of u
  aux = tuple(aux)  # Convert to tuple
  L = [(u[i], aux[i]) for i in range(len(u))]  # Create pairs of (u[i], -u[i])
  Z = list(itertools.product(*L))  # Generate all possible combinations of the pairs
  return np.array(Z)


def integrand_to_optimize_GBM_call_on_min(R):
    """Calculate the integrand of the GBM to optimize .
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): integrand value at the origin (u = 0)
    """
    d = len(S0)  # dimensionality
    X0 = np.log(np.divide(S0, K))
    y = np.multiply(1j, R)
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Characteristic function
    p = fourier_payoff_call_on_min(y)  # Fourier Transformed Payoff function
    discount = K * ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # modified discount factor
    integrand = discount * phi * p
    return np.real(integrand)  # Real part of the integrand


def integrand_to_optimize_GBM_basket_put(R):
    """Calculate the integrand of the GBM to optimize .
    Args:
    - R (array): Array of damping parameters.
    Returns:
    - integrand (float): integrand value at the origin (u = 0)
    """
    d = len(S0)  # dimensionality
    X0 = np.log(np.divide(S0, d*K))
    y = np.multiply(1j, R)
    phi = GBM_characteristic_function(y, sigma, SIGMA, T, r)  # Characteristic function
    p = fourier_payoff_basket_put(y)  # Fourier Transformed Payoff function
    discount = K * ((2 * np.pi) ** (-d)) * np.exp(-r * T) * np.exp(-R @ X0)  # modified discount factor
    integrand = discount * phi * p
    return np.real(integrand)  # Real part of the integrand

# Call on min

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [8]:
# Model and payoff parameters
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 3 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def rainbow_constraint_1(R):
  return -1*R
def rainbow_constraint_2(R):
  return -1 - np.sum(R)
cons = ( {'type': 'ineq', 'fun': rainbow_constraint_1},
        {'type': 'ineq', 'fun': rainbow_constraint_2},)
# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_call_on_min, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [-6.07088613 -6.07088612 -6.07088607]


## Pricing using Tensor Product Gauss-Laguerre quadrature in the Fourier space

In [11]:
############### Model and payoff parameters ###############
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 3 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### QMC parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
R_init = -2*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_call_on_min, constraints = cons, x0 = R_init , method = "trust-constr" )
R = optimal_R.x
LagQUAD_estimate = LagQUAD_fourier_GBM_call_on_min_pricer(S0, K, r, T, sigma, SIGMA, N, R)
print("LagQUAD estimate =", round(LagQUAD_estimate,5) )

LagQUAD estimate = 0.6511


# Basket Put

## Computing the damping parameters using the rule proposed in [link to the paper](https://arxiv.org/pdf/2203.08196.pdf)

In [15]:
 # Model and payoff parameters
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 3 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### Setting for the optimal damping parameters #############
# Constraints related to the strip of regularity of the payoff transform
def basket_put_constraint(R):
    return R
cons = ( {'type': 'ineq', 'fun': basket_put_constraint},)
# Characteristic function of GBM is an entire function hence there are no related constraints to it.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
#print(optimal_R) # uncomment to see wether the optimizer converged succesfully.
R = optimal_R.x
print("Optimal damping parameters:", R)

Optimal damping parameters: [4.57832758 4.57832758 4.57832759]


In [17]:
############### Model and payoff parameters ###############
K = 100 # strike price
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 3 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### QMC parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
R = optimal_R.x
LagQUAD_estimate = LagQUAD_fourier_GBM_basket_put_pricer(S0, K, r, T, sigma, SIGMA, N, R)
print("LagQUAD estimate =", round(LagQUAD_estimate,5) )

LagQUAD estimate = 4.63453


# 1D multi-strike vectorized implementations (Caching)

In [30]:
def multistrike_LagQUAD_fourier_GBM_vanilla_call_pricer(S0,K,r,T,sigma,SIGMA,N,R):
  # vectorized implementation of the pricer for Vanilla calls.
  K = np.array(K).reshape([len(K),1]) # list of strikes
  d = 1
  [u,w] = np.polynomial.laguerre.laggauss(N) #Laguerre quadrature nodes and weights.
  u_1d=[u for i in range(d)] #array of 1-D abcissas for all directions
  w_1d=[w for i in range(d)] #array of 1-D weights for all directions
  u = np.asarray(list(product(*u_1d))) # Full Tensor Isotropic Grid of abcissas
  w = np.asarray(list(product(*w_1d))) #Full Tensor Isotropic Grid of weights
  X0 = np.log(np.divide(S0,d*K)) #element-wise division X_0 is logarithm of stock price at initial time
  phi_list = np.zeros(N, dtype = complex) #List of characteristic function evaluations
  p_list = np.zeros(N, dtype = complex) #List of payoff function evaluations
  w_prod_list = np.zeros(N, dtype = float)
  reciprocal_weight_function = np.zeros(N, dtype = float)
  phi_values = np.array(list(map(lambda u: GBM_characteristic_function(u + 1j * R, sigma, SIGMA, T, r), u) ) )
  p_values = np.array(list(map(fourier_payoff_call_on_min, u+1j*R)))
  w_prod_values = np.prod(w,axis = 1)
  reciprocal_weight_function = np.exp(np.sum(u,axis = 1 ))
  temp = p_values * phi_values * w_prod_values * reciprocal_weight_function
  mat = np.exp(1j * np.outer(X0,u))
  K_factor = (2*np.pi)**(-d) * np.exp(-r*T) * K * np.exp(-R*X0)
  V = 2 * K_factor.reshape(K.shape[0],)  * np.real( mat @ temp ) # The factor 2 comes from evenness of the integrand
  return V

In [31]:
############### Model and payoff parameters ###############
K_grid = list(np.linspace(80,120,40)) # strike prices grid
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 1 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### QMC parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
K = np.median(K_grid)
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
R = optimal_R.x
LagQUAD_estimates = multistrike_LagQUAD_fourier_GBM_vanilla_call_pricer(S0,K_grid,r,T,sigma,SIGMA,N,R)
print("LagQUAD estimates =", LagQUAD_estimates )

LagQUAD estimates = [ 1.18592931  1.35274689  1.53602154  1.73651592  1.95494817  2.19198648
  2.44824428  2.72427613  3.02057426  3.33756594  3.67561151  4.03500319
  4.41596456  4.81865075  5.24314922  5.68948118  6.1576035   6.64741114
  7.15873997  7.69136996  8.24502866  8.81939492  9.4141028  10.0287456
 10.66287995 11.31602994 11.98769124 12.67733517 13.38441263 14.10835802
 14.84859287 15.60452946 16.37557417 17.16113065 17.96060281 18.77339762
 19.59892761 20.43661331 21.28588532 22.14618627]


# Multidimensional multistrike vectorized implementation (caching)

In [35]:
def phase_factor(u,X0):
  return np.exp(1j * u @ X0.T )

def multistrike_LagQUAD_fourier_GBM_call_on_min_pricer(S0,K,r,T,sigma,SIGMA,N,R):
  d=len(S0) #number of underlying stocks / dimensionality of the problem
  K = np.array(K).reshape([len(K),1]) # list of strikes to be valued simulatenously
  k = K.shape[0] # number of strikes to be valued
  [u,w] = np.polynomial.laguerre.laggauss(N) #returns Laguerre quadrature nodes and weights.
  u_1d=[u for i in range(d)] # Nxd array of 1D abcissas
  w_1d=[w for i in range(d)] # Nxd array of 1D weights
  u = np.asarray(list(product(*u_1d))) # N^d x d: array of N^d d-dimensional points
  w = np.asarray(list(product(*w_1d))) # N^d x d: array of N^d d-dimensional weights
  u_laguerre_scaling = np.repeat(a = u, repeats = 2**d ,axis = 0) # Repeats each element of the array 2**d times before moving to next element
  w = np.repeat(a = w, repeats = 2**d ,axis = 0) # Repeats each element of the array 2**d times before moving to next element
  u = np.vstack(np.array(list(map(laguerre_combinations, u)))) # creates all possible combinations of multiplying by -1.
  X0 = np.log(np.divide(S0,K)) #element-wise division X_0 is logarithm of stock price at initial time - Division by d is intrinsic to basket option for equally weighted average
  phi_values = np.array(list(map(lambda u: GBM_characteristic_function(u + 1j * R, sigma, SIGMA, T, r), u)), dtype = complex) # characteristic function evaluations
  p_values = np.array(list(map(fourier_payoff_call_on_min, u+1j*R)), dtype = complex) # payoff transform evaluations
  w_prod_values = np.prod(w,axis = 1) # contains product of weights for each multi-index
  reciprocal_weight_function = np.exp(np.sum(u_laguerre_scaling,axis = 1 ))  # contains the inverse of the weight function applied to use Gauss-Laguerre.
  temp =  p_values * phi_values * w_prod_values * reciprocal_weight_function
  mat = np.array(list(map(lambda u: phase_factor(u,X0), u)))
  K_factor = (2*np.pi)**(-d) * np.exp(-r*T) * K *  np.exp(-1*np.einsum("ij,ij->i", np.tile(R,(k,1)), X0)).reshape(k,1) # Einstein sumation allows for dot product row-by-row between two matrices. Tiling repeats the vector k times, axis = 1 is to make sure we copy rows.
  V = K_factor  * np.real(np.einsum("ij,ij->i", mat.T, np.tile(temp,(k,1)))).reshape(k,1)
  return V

In [37]:
############### Model and payoff parameters ###############
K_grid = list(np.linspace(80,120,40)) # strike prices grid
r = 0 # risk-free interest rate
T = 1 # maturity date
dimension = 3 # number of underlying assets
S0 = 100 * np.ones(dimension) # vector of spot prices
sigma = 0.2 * np.ones(dimension) # vector of volatilities
rho = np.identity(dimension) # correlation matrix
SIGMA = covariance_matrix(sigma,rho) # covariance matrix

############### QMC parameters ###############
N = 2**3 # number of Gauss-Laguerre quadrature points per dimension.
R_init = 1*np.ones(dimension) # initial parameters R needs to belong to the strip of analyticity of the integrand
K = np.median(K_grid)
optimal_R = optimize.minimize(fun = integrand_to_optimize_GBM_basket_put, constraints = cons, x0 = R_init , method = "trust-constr" )
R = optimal_R.x
LagQUAD_estimates = multistrike_LagQUAD_fourier_GBM_call_on_min_pricer(S0,K_grid,r,T,sigma,SIGMA,N,R)
print("LagQUAD estimates =", LagQUAD_estimates )

LagQUAD estimates = [[0.01132429]
 [0.01574754]
 [0.02161207]
 [0.02928836]
 [0.03921311]
 [0.05189419]
 [0.06791443]
 [0.08793412]
 [0.11269201]
 [0.14300453]
 [0.17976319]
 [0.22393012]
 [0.27653171]
 [0.33865038]
 [0.41141486]
 [0.49598885]
 [0.59355868]
 [0.70531996]
 [0.83246379]
 [0.97616272]
 [1.13755688]
 [1.31774054]
 [1.51774948]
 [1.73854942]
 [1.98102561]
 [2.24597403]
 [2.53409397]
 [2.84598244]
 [3.1821301 ]
 [3.54291905]
 [3.928622  ]
 [4.33940319]
 [4.77532059]
 [5.23632938]
 [5.72228658]
 [6.23295666]
 [6.76801779]
 [7.32706889]
 [7.90963699]
 [8.51518494]]
